# Module import

In [1]:
# import some common libraries
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import datetime, os
import time

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog, transforms, build_detection_test_loader, build_detection_train_loader
# from detectron2.engine import DefaultTrainer
# from detectron2.structures import BoxMode
from dataset.crack_images import get_cracks_dicts_instance

# Setting for dataset and model

## Model

In [3]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (crack)
# cfg.MODEL.WEIGHTS = "/home/aicenter/Documents/hsu/crack-detection/models/Mask R-CNN/HomeInspection/CC140+30+22+35+72_001/model_final.pth"
cfg.MODEL.WEIGHTS = "/home/aicenter/Documents/hsu/crack_temp/models/IPCSHM/Mask R-CNN/train_cropped_512/model_final.pth"
# cfg.MODEL.WEIGHTS = "/home/aicenter/Documents/hsu/crack-detection/models/Mask R-CNN/CC140+73+22_Augmented/model_final.pth"
# cfg.MODEL.WEIGHTS = "/home/aicenter/Documents/hsu/crack-detection/models/Mask R-CNN/ongoing_experiments/chungliao-finetune/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("cracks_test", )
predictor = DefaultPredictor(cfg)

In [28]:
# parameters
model_parameters = filter(lambda p: p.requires_grad, predictor.model.backbone.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

45516800

## Dataset

In [2]:
test_dir = '/home/aicenter/Documents/hsu/data/CECI_Project/chungliao/cropped_fixed_512/'

In [3]:
DatasetCatalog.clear()
DatasetCatalog.register("cracks_test", lambda d= "test": get_cracks_dicts_instance(test_dir))
MetadataCatalog.get("cracks_test").set(thing_classes=["crack"], thing_colors=[(96, 151, 255)])

namespace(name='cracks_test',
          thing_classes=['crack'],
          thing_colors=[(96, 151, 255)])

In [5]:
test_loader = build_detection_test_loader(cfg, "cracks_test")

# Data Visualization

In [43]:
def draw_segmentation_single_image(base_image, predictor, input_image=np.array([])):
    if not input_image.any():
        input_image = base_image
    outputs = predictor(input_image)
    masks = outputs['instances'].pred_masks.cpu().numpy()
    masks = masks[outputs['instances'].scores.cpu().numpy() > cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST]
    mask = np.sum(masks, axis=0) > 0
    roi = base_image[mask]
    base_image[mask] = ((0.4 * mask_color) + (0.6 * roi)).astype("uint8")
    return base_image

def draw_segmentation_images(base_image, crop_size, predictor, color=np.array((0, 255, 0), dtype="uint8"), input_image=np.array([])):
    if not input_image.any():
        input_image = base_image
    height, width, _ = base_image.shape 
    crop_width, crop_height = crop_size
    x_num = int(width/crop_width) + 1
    y_num = int(height/crop_height) + 1
    max_x = width - crop_width
    max_y = height - crop_height
    for x in range(x_num):
        for y in range(y_num):
            start_x = min(x*crop_width, max_x)
            end_x = start_x + crop_width
            start_y = min(y*crop_height, max_y)
            end_y = start_y + crop_height
            _input = input_image[start_y:end_y, start_x:end_x, :]
            _base = base_image[start_y:end_y, start_x:end_x, :]
            _base = draw_segmentation_single_image(_base, predictor, _input)
            base_image[start_y:end_y, start_x:end_x, :] = _base
    return base_image

def draw_segmentation_and_bbox_single_image(base_image, predictor, meta_dataset, input_image=np.array([])):
    vis_metadata = MetadataCatalog.get(meta_dataset)
    if not input_image.any():
        input_image = base_image
    outputs = predictor(input_image)
    v = Visualizer(base_image[:, :, ::-1], metadata=vis_metadata, scale=1, instance_mode=ColorMode.SEGMENTATION)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    return v.get_image()[:, :, ::-1]


def draw_segmentation_and_bbox_images(base_image, crop_size, predictor, meta_dataset, input_image=np.array([])):
    if not input_image.any():
        input_image = base_image
    height, width, _ = input_image.shape 
    crop_width, crop_height = crop_size
    x_num = int(width/crop_width) + 1
    y_num = int(height/crop_height) + 1
    max_x = width - crop_width
    max_y = height - crop_height
    for x in range(x_num):
        for y in range(y_num):
            start_x = min(x*crop_width, max_x)
            end_x = start_x + crop_width
            start_y = min(y*crop_height, max_y)
            end_y = start_y + crop_height
            _input = input_image[start_y:end_y, start_x:end_x, :]
            _base = draw_segmentation_and_bbox_single_image(_input, predictor, meta_dataset)
            base_image[start_y:end_y, start_x:end_x, :] = _base
    return base_image


def save_image(path, im):
    print('Saved at', path)
    cv2.imwrite(path, im)
    

* ## Labels

In [20]:
save_dir = test_dir + '/gt_visualize/'
os.makedirs(save_dir, exist_ok=True)

mask_color = np.array((43, 255, 244), dtype="uint8")
for idx, d in enumerate(test_loader):
    file_name = d[0]["file_name"]
    im = cv2.imread(file_name)
    if idx%50==0:
        print(file_name)
    
    image_name = file_name.split('/')[-1]
    masks = cv2.imread(d[0]['seg_file_name'], cv2.IMREAD_GRAYSCALE)
    mask = masks == 255
    roi = im[mask]
    im[mask] = ((0.5 * mask_color) + (0.5 * roi)).astype("uint8")
    cv2.imwrite(save_dir + file_name.split('/')[-1], im)

/home/aicenter/Documents/hsu/data/CECI_Project/chungliao/cropped_fixed_512/image/001_01.png
/home/aicenter/Documents/hsu/data/CECI_Project/chungliao/cropped_fixed_512/image/001_71.png


* ## Segmentation: original size

In [12]:
# image_paths = ['/home/aicenter/Documents/hsu/data/test_dataset/test_tunnel/T3/t3_tunnel.png']
root = '/home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/test/'
root = '/home/aicenter/Documents/hsu/data/project_sinotech/source/HS-N_19K20K_half_01/'
root = '/home/aicenter/Documents/hsu/data/publication_IPC-SHM/test/image/'
root = '/home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/'
image_paths = [root + file for file in os.listdir(root)]

***Mode 1: input of the whole picture and draw segmentation on images***

In [38]:
os.makedirs(root + '../seg/', exist_ok=True)
for image_path in image_paths:
    base_image = cv2.imread(image_path)
    input_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    input_image = cv2.cvtColor(input_image, cv2.COLOR_GRAY2BGR)
    height, width, _ = input_image.shape 
    crop_size = (512, 512)
    base_image = draw_segmentation_single_image(base_image, crop_size, predictor, input_image=input_image)
    save_image(root + '../seg/' + image_path.split('/')[-1][:-4] + '_seg.png', base_image)

Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../seg/DSC09612_seg.png
Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../seg/DSC09666_seg.png
Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../seg/DSC09652_seg.png
Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../seg/DSC09620_seg.png


***Mode 2: input of the whole picture and draw only segmentation***

In [40]:
os.makedirs(root + '../seg_only/', exist_ok=True)
for image_path in image_paths:
    filename = image_path.split('/')[-1]
    input_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    input_image = cv2.cvtColor(input_image, cv2.COLOR_GRAY2BGR)
    height, width, _ = input_image.shape
    base_image = np.zeros((height, width, 3))  
    crop_size = (512, 512)
    
    base_image = draw_segmentation_images(base_image, crop_size, predictor, input_image=input_image)
    
    kernel = np.ones((3,3), np.uint8)
    dilation = cv2.dilate(base_image, kernel, iterations = 5)
    mask = np.sum(dilation, axis=2) > 0
    base_image = np.ones((height, width, 3))*255    
    roi = base_image[mask]
    base_image[mask] = ((1 * np.array((0, 255, 0), dtype="uint8") + (0 * roi))).astype("uint8")
    save_image(root + '../seg_only/' + filename[:-4] + '.png', base_image)

Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../seg_only/DSC09612.png
Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../seg_only/DSC09666.png
Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../seg_only/DSC09652.png
Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../seg_only/DSC09620.png


***Mode 3: input of the whole picture and seg only predicted parts overlap to gt***

In [ ]:
os.makedirs(root + '../overlap/', exist_ok=True)
for image_path in image_paths:
    filename = image_path.split('/')[-1]
    input_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    input_image = cv2.cvtColor(input_image, cv2.COLOR_GRAY2BGR)
    height, width, _ = input_image.shape 
    base_image = np.zeros((height, width, 3))  
    
    crop_size = (512, 512)
    base_image = draw_segmentation_images(base_image, crop_size, predictor, input_image=input_image)
    
    kernel = np.ones((3,3), np.uint8)
    dilation = cv2.dilate(base_image, kernel, iterations = 5)
    mask = np.sum(dilation, axis=2) > 0
    base_image = cv2.imread(root + '../gt/' + filename[:-4] + '.png')
    base_image = cv2.resize(base_image, (width, height), interpolation=cv2.INTER_CUBIC)
    roi = base_image[mask]
    base_image[mask] = ((1 * np.array((0, 255, 0), dtype="uint8") + (0 * roi))).astype("uint8")
    
    save_image(root + '../overlap/' + filename[:-4] + '_overlap.png', base_image)

* ## Segmentation: cropped size

***Mode 1: do image stitching***

In [ ]:
### Mode 1: input with cropped image and do image stitching
crop_w, crop_h = 256, 256
ceci_dir = '/home/aicenter/Documents/hsu/data/test_dataset/test_ceci/640xh/%sx%s/image/' %(crop_w, crop_h)
source_path = '/home/aicenter/Documents/hsu/data/test_dataset/test_ceci/640xh/gt_640.png'

height, width, _ = cv2.imread(source_path).shape
x_num, y_num = int(width/crop_w), int(height/crop_h)

mask_color = np.array((0, 0, 255), dtype="uint8") 
whole_image = np.array([])
v_im = np.array([])
for filename in sorted(os.listdir(ceci_dir)):
    count_id = int(filename.split('.')[0])
    image_path = ceci_dir+filename
    im = cv2.imread(image_path)
    im = run_prediction_and_draw_single_image(im, predictor)
    
    if (count_id+1)%y_num == 0:
        whole_image = v_im if whole_image.size == 0 else np.hstack((whole_image, v_im))
        v_im = np.array([])
    else:
        v_im = im if v_im.size == 0 else np.vstack((v_im, im))

save_image(source_path.split('.png')[0] + '_seg.png', whole_image)

* ## Box+Segmentation: original size

In [44]:
# image_paths = ['/home/aicenter/Documents/hsu/data/test_dataset/test_tunnel/3yi_tunnel.png']
# root = '/home/aicenter/Documents/hsu/data/test_dataset/test_tunnel/CL/img/'
# root = '/home/aicenter/Documents/hsu/data/test_dataset/test_tunnel/south/crop/'
root = '/home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/test/'
root = '/home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/'
image_paths = [root + file for file in os.listdir(root)]

In [45]:
os.makedirs(root + '../bbox/', exist_ok=True)
for image_path in image_paths:
    filename = image_path.split('/')[-1]
    base_image = cv2.imread(image_path)
    input_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    input_image = cv2.cvtColor(input_image, cv2.COLOR_GRAY2BGR)
    crop_size = (512, 512)
    base_image = draw_segmentation_and_bbox_images(base_image, crop_size, predictor, "cracks_test", input_image=input_image )
         
    save_image(root + '../bbox/' + filename[:-4] + '_bbox.png', base_image)

Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../bbox/DSC09612_bbox.png
Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../bbox/DSC09666_bbox.png
Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../bbox/DSC09652_bbox.png
Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../bbox/DSC09620_bbox.png


* ## Box+Segmentation: cropped size

In [10]:
save_dir = test_dir + '/result/'
os.makedirs(save_dir, exist_ok=True)

for idx, d in enumerate(test_loader):
    file_name = d[0]["file_name"]
    base_image = cv2.imread(file_name)
    base_image = draw_segmentation_and_bbox_single_image(base_image, predictor, "cracks_test")
    if idx%50==0:
        print(file_name)
    image_name = file_name.split('/')[-1]
    cv2.imwrite(save_dir + file_name.split('/')[-1], base_image)

/home/aicenter/Documents/hsu/data/IPC-SHM-P1/experiments/val_120+80/image/001.png


# Evaluation

* ## mIoU 

In [6]:
# borrows from pytorch-deeplab-xception
from utils.metrics import Evaluator
evaluator = Evaluator(2)

In [9]:
is_trans = 'Trans' in test_dir
root, folder_name = '', ''
if is_trans:
    split_path = file_name.split('/')
    folder_name = '_'.join(split_path[-3].split('_')[:-1])
    root = '/'.join(split_path[:-3])
    
evaluator.reset()
for idx, d in enumerate(test_loader):
    file_name = d[0]["file_name"]
    im = cv2.imread(file_name)
    if idx%50==0:
        print(file_name)
    outputs = predictor(im)
    
    image_name = file_name.split('/')[-1]
    if is_trans:
        original_path = '%s/%s/image/%s'%(root, folder_name, image_name)
        im = cv2.imread(original_path)
    masks = outputs['instances'].pred_masks.cpu().numpy()
    masks = masks[outputs['instances'].scores.cpu().numpy() > cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST]
    mask = np.sum(masks, axis=0) > 0
    target = cv2.imread(d[0]['seg_file_name'], cv2.IMREAD_GRAYSCALE)
    evaluator.add_batch(target/255, mask)

MIoU = np.diag(evaluator.confusion_matrix) / (
    np.sum(evaluator.confusion_matrix, axis=1) + np.sum(evaluator.confusion_matrix, axis=0) -
    np.diag(evaluator.confusion_matrix))
Acc = evaluator.Pixel_Accuracy()
Acc_class = evaluator.Pixel_Accuracy_Class()
mIoU = evaluator.Mean_Intersection_over_Union()
FWIoU = evaluator.Frequency_Weighted_Intersection_over_Union()
print('Validation:')
print('mIoU:{}'.format(MIoU))
print("Acc:{}, Acc_class:{}, mIoU:{}, fwIoU: {}".format(Acc, Acc_class, mIoU, FWIoU))

/home/aicenter/Documents/hsu/data/CECI_Project/chungliao/cropped_fixed_512/image/001_01.png
/home/aicenter/Documents/hsu/data/CECI_Project/chungliao/cropped_fixed_512/image/001_71.png
Validation:
mIoU:[0.9912531  0.08246476]
Acc:0.9912599690755208, Acc_class:0.5720960407605691, mIoU:0.5368589284189166, fwIoU: 0.9864442041603741


* ## mAP計算

In [63]:
test_dir = '/home/aicenter/Documents/hsu/data/CECI_Project/chungliao/val_cropped_fixed_512/'
DatasetCatalog.clear()
DatasetCatalog.register("cracks_test", lambda d= "test": get_cracks_dicts_instance(test_dir))
MetadataCatalog.get("cracks_test").set(thing_classes=["crack"], thing_colors=[(96, 151, 255)])
test_loader = build_detection_test_loader(cfg, "cracks_test")

In [72]:
from detectron2.evaluation import COCOEvaluator
evaluator = COCOEvaluator("cracks_test", cfg, True, 'coco_eval')

In [73]:
evaluator.reset()

for idx, d in enumerate(test_loader):
    file_name = d[0]["file_name"]
    im = cv2.imread(file_name)
    if idx%50==0:
        print(file_name)
    outputs = predictor(im)
    evaluator.process(d, [outputs])
    
evaluator.evaluate()

/home/aicenter/Documents/hsu/data/CECI_Project/chungliao/val_cropped_fixed_512/image/001_40.png
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
COCOeval_opt.evaluate() finished in 0.00 seconds.
Accumulating evaluation results...
COCOeval_opt.accumulate() finished in 0.00 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.169
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.314
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.166
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.208
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.156
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | max

OrderedDict([('bbox',
              {'AP': 16.91559389455429,
               'AP50': 31.374405297672624,
               'AP75': 16.58040804080408,
               'APs': 20.759427642573737,
               'APm': 12.517680339462517,
               'APl': nan}),
             ('segm',
              {'AP': 0.0,
               'AP50': 0.0,
               'AP75': 0.0,
               'APs': 0.0,
               'APm': 0.0,
               'APl': nan})])

# Geometric Properties Calculation

In [302]:
def get_dist(x1, y1, x2, y2):
    return ((x2-x1)**2 + (y1-y2)**2)**0.5

def is_merged(box_1, box_2, threshold=100):
    left_1, top_1, right_1, bottom_1 = box_1
    left_2, top_2, right_2, bottom_2 = box_2
    if left_1 > left_2 and right_1 < right_2:
        if top_1 > top_2 and bottom_1 < bottom_2:
            return True
    coner_1 = [(right_1, bottom_1), (right_1, top_1), (left_1, bottom_1), (left_1, top_1)]
    coner_2 = [(right_2, bottom_2), (right_2, top_2), (left_2, bottom_2), (left_2, top_2)]
    # cal minimum distance between coner points
    val = 10**8
    for c1 in coner_1:
        for c2 in coner_2:
            val = min(val, get_dist(c1[0], c1[1], c2[0], c2[1]))                      
    return val < threshold

In [287]:
def draw_lines(image, points):
    for i in range(len(points)):
        min_index, min_val = -1, 10**8
        for j in range(i+1, len(points)):
            x1, y1 = points[i]
            x2, y2 = points[j]
            dist = get_dist(x1, y1, x2, y2)
            if dist <= 1.1:
                min_index = j
                break
            if min_val > dist:
                min_val = dist
                min_index = j
        if min_index != -1:
            cv2.line(image, points[i], points[min_index], (255,255,255), 1)
    return image

In [304]:
crack_metadata = MetadataCatalog.get("cracks_test")

root = '/home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/'
# root = '/home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/test/'
crack_image_paths = [root + file for file in os.listdir(root)]

os.makedirs(root + '../bbox/', exist_ok=True)

for image_path in crack_image_paths:
    filename = image_path.split('/')[-1]
    crack_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    crack_image = cv2.cvtColor(crack_image, cv2.COLOR_GRAY2BGR)
    height, width, _ = crack_image.shape 
    crop_width, crop_height = (512, 512)
    mask_color = np.array((0, 255, 0), dtype="uint8") 
    x_num = int(width/crop_width) + 1
    y_num = int(height/crop_height) + 1
    max_x = width - crop_width
    max_y = height - crop_height
    
    box_list = []
    mask_list = []
    ref_pts = []
    for x in range(x_num):
        for y in range(y_num):
            start_x = min(x*crop_width, max_x)
            end_x = start_x + crop_width
            start_y = min(y*crop_height, max_y)
            end_y = start_y + crop_height
            test_image = crack_image[start_y:end_y, start_x:end_x, :]
            
            outputs = predictor(test_image)
            bboxes = outputs['instances'].pred_boxes.tensor.cpu().numpy()
            bboxes = bboxes[outputs['instances'].scores.cpu().numpy() > 0.5]
            if len(bboxes) == 0:
                continue
                
            masks = outputs['instances'].pred_masks.cpu().numpy()
            masks = masks[outputs['instances'].scores.cpu().numpy() > 0.5]
            
            for bbox, mask in zip(bboxes, masks):
                left, top, right, bottom = list(map(int, bbox))
                left += start_x
                top += start_y
                right += start_x
                bottom += start_y
#                 if (right-left)*(bottom-top)/width/height < 4e-4:
#                     continue
                box_list.append([left, top, right, bottom])
                mask_list.append(mask)
                ref_pts.append((start_x, end_x, start_y, end_y))
                
    if box_list != []:
        box_list, mask_list, ref_pts = zip(*sorted(zip(box_list, mask_list, ref_pts), key=lambda x: x[0]))
        mask_groups = [[0]]
        box_groups = [[box_list[0]]]
        for i in range(1, len(box_list)):
            candidate = box_list[i]
            ismatch = False
            for gindex in range(len(box_groups)):
                if ismatch:
                    break
                for box in box_groups[gindex]:
                    if is_merged(candidate, box, 30):
                        box_groups[gindex].append(candidate)
                        mask_groups[gindex].append(i)
                        ismatch = True
                        break
            if not ismatch:
                box_groups.append([candidate])
                mask_groups.append([i])

        colors = [plt.cm.plasma(i/float(len(box_groups))) for i in range(len(box_groups))]
        for num, (boxes, mask_indexes) in enumerate(zip(box_groups, mask_groups)):
            color = list(map(lambda x:int(x*255), colors[num][:3]))
            
            output_left, output_top, output_right, output_bottom = 10**8, 10**8, -1, -1
            boxes = sorted(boxes, key=lambda x: x[0])
            im = np.zeros((height, width))*255
            
            for box, index in zip(boxes, mask_indexes):
                left, top, right, bottom = box
                output_left = min(output_left, left)
                output_top = min(output_top, top)
                output_right = max(output_right, right)
                output_bottom = max(output_bottom, bottom)
                
                
                mask = mask_list[index]
                start_x, end_x, start_y, end_y = ref_pts[index]
                im[start_y:end_y, start_x:end_x] += mask
                
            cv2.rectangle(crack_image, (output_left, output_top), (output_right, output_bottom), color, 5)
            
            mask = im > 0
            points = list(np.argwhere(mask))
            if len(points) >= 2:
                points = sorted(points, key=lambda x: x[1])
                slope = (points[0][0]-points[-1][0])*(points[0][1]-points[-1][1])
                if slope < 0: #座標軸為向下向右正
                    cv2.line(crack_image, (output_left, output_bottom), (output_right, output_top), color, 1)
                else:
                    cv2.line(crack_image, (output_left, output_top), (output_right, output_bottom), color, 1)
            color = np.array(color, dtype="uint8") 
            roi = crack_image[mask]
            crack_image[mask] = ((0.4 * color) + (0.6 * roi)).astype("uint8")
            
    
    
    save_path = root + '../bbox/' + filename[:-4] + '.png'
    print('Saved at', save_path)
    cv2.imwrite(save_path, crack_image)
#     break

Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../bbox/DSC09612.png
Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../bbox/DSC09666.png
Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../bbox/DSC09652.png
Saved at /home/aicenter/Documents/hsu/data/project_home_inspection/cracks/exp_combination/measure_sample/../bbox/DSC09620.png


# IPC-SHM Draw Cropped Segmentation

In [17]:
size = 512

In [ ]:
# borrows from pytorch-deeplab-xception
from utils.metrics import Evaluator
evaluator = Evaluator(2)

In [ ]:
is_trans = 'Trans' in test_dir
root, folder_name = '', ''
if is_trans:
    split_path = file_name.split('/')
    folder_name = '_'.join(split_path[-3].split('_')[:-1])
    root = '/'.join(split_path[:-3])
    
evaluator.reset()
for idx, d in enumerate(test_loader):
    file_name = d[0]["file_name"]
    im = cv2.imread(file_name)
    if idx%50==0:
        print(file_name)
    outputs = predictor(im)
    
    image_name = file_name.split('/')[-1]
    if is_trans:
        original_path = '%s/%s/image/%s'%(root, folder_name, image_name)
        im = cv2.imread(original_path)
    masks = outputs['instances'].pred_masks.cpu().numpy()
    masks = masks[outputs['instances'].scores.cpu().numpy() > cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST]
    mask = np.sum(masks, axis=0) > 0
    target = cv2.imread(d[0]['seg_file_name'], cv2.IMREAD_GRAYSCALE)
    evaluator.add_batch(target/255, mask)

MIoU = np.diag(evaluator.confusion_matrix) / (
    np.sum(evaluator.confusion_matrix, axis=1) + np.sum(evaluator.confusion_matrix, axis=0) -
    np.diag(evaluator.confusion_matrix))
Acc = evaluator.Pixel_Accuracy()
Acc_class = evaluator.Pixel_Accuracy_Class()
mIoU = evaluator.Mean_Intersection_over_Union()
FWIoU = evaluator.Frequency_Weighted_Intersection_over_Union()
print('Validation:')
print('mIoU:{}'.format(MIoU))
print("Acc:{}, Acc_class:{}, mIoU:{}, fwIoU: {}".format(Acc, Acc_class, mIoU, FWIoU))

In [18]:
save_dir = test_dir + '/result/'
os.makedirs(save_dir, exist_ok=True)

mask_color = np.array((0, 255, 0), dtype="uint8")
for idx, d in enumerate(test_loader):
    file_name = d[0]["file_name"]
    im = cv2.imread(file_name)
    if idx%50==0:
        print(file_name)
    height, width, _ = im.shape
    h_split_number = int(height/size)
    w_split_number = int(width/size)
    whole_image = np.array([])
    v_im = np.array([])
    for w_num in range(w_split_number):
        start_x = w_num*size
        end_x = start_x + size
        for h_num in range(h_split_number):
            start_y = h_num*size
            end_y = start_y + size
            cropped_image = im[start_y:end_y, start_x:end_x, :]
            outputs = predictor(cropped_image)
            masks = outputs['instances'].pred_masks.cpu().numpy()
            masks = masks[outputs['instances'].scores.cpu().numpy() > cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST]
            mask = np.sum(masks, axis=0) > 0
            roi = cropped_image[mask]
            cropped_image[mask] = ((0.5 * mask_color) + (0.5 * roi)).astype("uint8")
            v_im = cropped_image if v_im.size == 0 else np.vstack((v_im, cropped_image))
        
        whole_image = v_im if whole_image.size == 0 else np.hstack((whole_image, v_im))
        v_im = np.array([])
    cv2.imwrite(save_dir + file_name.split('/')[-1], whole_image)

/home/aicenter/Documents/hsu/data/IPC-SHM-P1/experiments/val_120+80/image/001.png


In [19]:
im = cv2.imread('/home/aicenter/Documents/hsu/data/test_dataset/test_tunnel/south/img/SN_437475000_437500000.tif')
crop_im = im[350:-1750, 40:-110, :]
cv2.imwrite('/home/aicenter/Documents/hsu/data/test_dataset/test_tunnel/south/img/SN_437475000_437500000_crop.png', crop_im)

True